In [ ]:
#source = https://techpolicyinstitute.org/publications/miscellaneous/rotten-tomatoes-and-imdb-reviews-strongly-correlated-with-movie-revenues/

## Scraping Rottentomatoes: https://www.rottentomatoes.com/browse/movies_in_theaters/

In [1]:
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import duckdb
from urllib import parse
from urllib.parse import urljoin

In [2]:
#create fake user agents to use while requesting
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]

In [3]:
def convert_word(word):
    """
    Strips a word from all punctuation, whitespace, and digits. Then converts the word into all lower case
    and convert all characters with accents to non-accent letters.
    """
    word = re.sub(r'[^\w\s]','',word).lower()
    word = re.sub(r'[àáâãäå]', 'a', word)
    word = re.sub(r'[èéêë]', 'e', word)
    word = re.sub(r'[ìíîï]', 'i', word)
    word = re.sub(r'[òóôõö]', 'o', word)
    word = re.sub(r'[ùúûü]', 'u', word)
    
    return word

In [4]:
def map_cleaned_strings(dfname, columnname):
    """
    Maps new values in a df-column. From a new created dictionary, with as Key the 'old' string value and 
    as Value the 'new'-string value. The 'new'-string values are extracted with the function convert_word().
    """
    dict_old_new = {}
    for i in dfname[columnname]:
        if type(i) != str:
            continue
        else:
            dict_old_new[i] = convert_word(i)

    dfname[columnname] = dfname[columnname].map(dict_old_new)
    
    return dfname[columnname]

### Get pagination

In [5]:
#get request to the first page url
url = 'https://www.rottentomatoes.com/browse/movies_at_home/?page=2'
# url = "https://www.rottentomatoes.com/browse/movies_at_home/?page=47"

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
# print(r.text)
soup = BeautifulSoup(r.content, "html.parser")


In [6]:
# Get all pages id (after click load more button)
urls = ['https://www.rottentomatoes.com/browse/movies_at_home/']
url = 'https://www.rottentomatoes.com/browse/movies_at_home/'
for count in range(2,100,1):                    ###### Don't know exactly how much there are?? try and check
    count = '?page=' + str(count)
    url = urljoin(url, count)
    urls.append(url)
# urls

In [7]:
# Get ids for the movie pages (after click on movie)
movie_pages = []
pages = soup.find_all('a', attrs = {'data-qa':'discovery-media-list-item-caption'}) 
# print(pages)

for p in pages:
    link_id = p.get('href')
    m_link = urljoin(url, link_id)
#     print(m_link)
    movie_pages.append(m_link)
# movie_pages

In [47]:
# Scrape info from movie pages

#get request to an example movie page url
url = 'https://www.rottentomatoes.com/m/tomorrows_hope'

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
soup = BeautifulSoup(r.content, "html.parser")

## get movie information (labels and values)
# create empty df
df_rot = pd.DataFrame()

#Get labels
label_col =[]
labels = soup.find_all('b', attrs = {'data-qa': 'movie-info-item-label'})
for l in labels:
    label_col.append(l.text.strip())

df_rot['labels'] = label_col
# label_col

#Get values of labels
val =[]
values = soup.find_all('span', attrs = {'data-qa': 'movie-info-item-value'})
for v in values:
    val.append(v.text.strip())
    
df_rot['val'] = val

# val[2]
# print(val)
# print(labels)




In [169]:
# Scrape info from movie pages

#get request to an example movie page url
url = 'https://www.rottentomatoes.com/m/tomorrows_hope'

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
soup = BeautifulSoup(r.content, "html.parser")

## get movie information (labels and values)
# create empty df
df_rot = pd.DataFrame()
d = dict()
lab = {}

#Get labels
label_col =[]
title = soup.find_all(class_="scoreboard__title")
# for i in title:
#     print(i.text)
labels = soup.find_all('b', attrs = {'data-qa': 'movie-info-item-label'})
values = soup.find_all('span', attrs = {'class':'info-item-value','data-qa': 'movie-info-item-value'})
# values2 = soup.find_all(class_="info-item-value")

# get values

for k, v in enumerate(values):
    for s in v:
        print(s)
    break

#     d[k] = v.text.strip()
# #         t.append([v.text])

# d
# # get labels
# for i, l in enumerate(labels):
#     df2 = df.assign(columns = [l.text.strip()])
# df2
#     lab[i] = l.text.strip()
    
# lab


# t[1]

# for l in labels:
#     for v in values2:
#         d[l.text.strip()] = v.text.strip()
# d

# df_rot['labels'] = label_col
# label_col

# #Get values of labels
# val =[]
# values = soup.find_all('span', attrs = {'data-qa': 'movie-info-item-value'})
# for v in values:
#     val.append(v.text.strip())
    
# df_rot['val'] = val

# val[2]
# print(val)
# print(labels)





                            
                            Documentary
                            
                        


In [153]:
pd.DataFrame.from_dict(d, orient='index', columns=['movie title'])


,movie title
0,Documentary
1,English
2,Thomas A. Morgan
3,"Tamra Raven, \n ..."
4,"May 20, 2021\n limited"
5,"Mar 21, 2023"
6,45m
7,Abramorama


In [48]:
df_rot

,labels,val
0,Genre:,Documentary
1,Original Language:,English
2,Director:,Thomas A. Morgan
3,Producer:,"Tamra Raven, \n ..."
4,Release Date (Theaters):,"May 20, 2021\n limited"
5,Release Date (Streaming):,"Mar 21, 2023"
6,Runtime:,45m
7,Distributor:,Abramorama


## Scraping all pages

In [177]:
### PUT IT ALL TOGETHER
df = pd.DataFrame()
lab_list = []
titles = []
val = []

for url in movie_pages:
    #get request to page
    r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    
    # request movie page from url
    for u in urls:
        req = requests.get(u, headers={'User-Agent': random.choice(user_agents_list)})
        soup = BeautifulSoup(req.content, "html.parser")
        
#         print(soup.prettify())
        ## get movie information (labels and values)

        #Get title, labels and values
        title = soup.find_all(class_="scoreboard__title")
        labels = soup.find_all('b', attrs = {'data-qa': 'movie-info-item-label'})
        values = soup.find_all('span', attrs = {'class':'info-item-value','data-qa': 'movie-info-item-value'})
        # values2 = soup.find_all(class_="info-item-value")


        # get labels and put them to the columns of the df
        for lab in labels:
            for l in lab:
                lab_list.append(l)
            df['labels'] = lab_list
#             df2 = df.assign(columns = [l.text.strip()])
#             lab[i] = l.text.strip()
#             l_df = pd.DataFrame.from_dict(d, orient='index', columns=['movie title'])
            
    
        # get title
        for t in title:
            titles.append(t.text)
        df['title'] = titles
            
        # get values
        for v in values:
            val.append(v.text.strip())
        df['values'] = val
#             v_df = pd.DataFrame.from_dict(d, orient='index', columns=['movie title'])
#             #join the labels df l_df
#             l_df = l_df.join(v_df)
        
#         df_from_lists = pd.DataFrame(
#             {'title': titles,
#              'labels': lab_list,
#              'values': val
#             })
        s1=pd.Series(titles,name='titles')
        s2=pd.Series(lab_list,name='labels')
        s3=pd.Series(val ,name='values')
        
        df2 = pd.concat([s1,s2,s3], axis=1)

        val = []
        titles = []
        lab_list= []



/var/folders/p3/3k9f5xrj7wn3f78wkf2507280000gn/T/ipykernel_12635/1653547464.py:54: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s1=pd.Series(titles,name='titles')
/var/folders/p3/3k9f5xrj7wn3f78wkf2507280000gn/T/ipykernel_12635/1653547464.py:55: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2=pd.Series(lab_list,name='labels')
/var/folders/p3/3k9f5xrj7wn3f78wkf2507280000gn/T/ipykernel_12635/1653547464.py:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s3=pd.Series(val ,name='values')


In [179]:
df2

,titles,labels,values


In [46]:
# Scrape info from movie pages

#get request to an example movie page url
url = 'https://www.rottentomatoes.com/m/tomorrows_hope'

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
soup = BeautifulSoup(r.content, "html.parser")

## get movie information (labels and values)
# create empty df with the columnnames
df_rot = pd.DataFrame(columns = ['Genre:', 'Original Language:', 'Director:', 'Producer:',
       'Release Date (Theaters):', 'Release Date (Streaming):', 'Runtime:',
       'Distributor:'])
rot = []

#Get labels
labels = soup.find_all('b', attrs = {'data-qa': 'movie-info-item-label'})
values = soup.find_all('span', attrs = {'data-qa': 'movie-info-item-value'})

for l in labels:
    for v in values:
        for item in [v.text]:
            row = [str(item).strip()]
            rot.append(row)
df = pd.DataFrame(rot, columns=['Genre:', 'Original Language:', 'Director:', 'Producer:',
'Release Date (Theaters):', 'Release Date (Streaming):', 'Runtime:',
'Distributor:'])
print(df)
            
      
            
            
# #             print('1.',item)
# #         print(v.text.strip())
#             rot_dict[str(l.text).strip()] = (str(item).strip())
# # print(rot_dict)
#             df = pd.DataFrame([rot_dict])
# #             df_rot = pd.concat([df_rot, df], ignore_index=True)
# #             df_rot
# rot_dict = {}


# Insert row to the dataframe using DataFrame.append()
# df = pd.DataFrame(technologies)
# new_row = {'Courses':'Hyperion', 'Fee':24000, 'Duration':'55days', 'Discount':1800}
# df2 = df.append(new_row, ignore_index=True)

ValueError: 8 columns passed, passed data had 1 columns

In [84]:
df_rot = df_rot.set_index('labels').T.reset_index()


In [85]:
df_rot.columns

Index(['index', 'Genre:', 'Original Language:', 'Director:', 'Producer:',
       'Release Date (Theaters):', 'Release Date (Streaming):', 'Runtime:',
       'Distributor:'],
      dtype='object', name='labels')

In [ ]:
### Next step is to add to dict or tuples and then to df?

In [ ]:
### Final step is when all above works as expected, put it all togheter and scrape every movie page of 
### every loaded page (load button)

### Clean names so we can merge with the project data

In [ ]:
#to do

### Add as table to database

In [ ]:
##to do
# # create a connection to a file called 'imdb_reviews.db'
# con = duckdb.connect('imdb_reviews.db')

# # create a table and load data into it
# con.sql("CREATE TABLE IF NOT EXISTS budget_data AS SELECT * FROM df")

# con.close()